# Simple approach to getting data

In this notebook we walk through a slightly simpler method of querying OOI 
data, where you don't need to know quite so many details to access the data.
 This methodology is from [the ooi repo](https://github.com/oceanobservatories/ooi-data-explorations/tree/master/python)
 and it includes a summary yaml file that simplifies and groups more than 
 100 nodes in the network in a way that makes queries more straightforward 
 for most users. 
 
Rather than defining site, node, sensor, method, and stream, users only need
 to specify site, assembly, instrument, and method.

In [1]:
from yooink.request.data_fetcher import DataFetcher
from yooink.utils import ooi_seconds_to_datetime
import os

In [2]:
# Initialize the API client with your username and token.
# 
# This assumes you have your OOI API information stored as environment 
# variables "OOI_USER" and "OOI_TOKEN". Adjust this as needed! You can 
# hard-code these but do this with caution (ie don't push to a public repo!)
username = os.getenv('OOI_USER')
token = os.getenv('OOI_TOKEN')

In [3]:
# Example setup for requesting pH sensor data from an instrument on the Oregon
# shelf surface mooring. 
site = 'ce02shsm'           # OOI site designator
assembly = 'midwater'       # Assembly grouping name
instrument = 'phsen'        # OOI instrument class 
method = 'telemetered'      # data delivery method

# You need to specify at least a start date and/or
# stop date, or use the deployment number
start = '2019-04-01'       # data for April 2019 ...
stop = '2019-05-30'        # ... through May 2019
deploy = 9                 # The Spring 2019 deployment number

In [4]:
fetcher = DataFetcher(username=username, token=token)
ds = fetcher.get_dataset(
    site=site,
    assembly=assembly,
    instrument=instrument,
    method=method,
    start=start,
    stop=stop,
)

Using cached URL for request: CE02SHSM_RID26_06-PHSEND000_telemetered_phsen_abcdef_dcl_instrument_2019-04-01T04:00:00.000Z_2019-05-30T04:00:00.000Z


Processing files: 100%|██████████| 2/2 [00:02<00:00,  1.16s/it]


In [5]:
# The result is an xarray dataset
ds

<xarray.Dataset> Size: 926kB
Dimensions:                                  (time: 1339,
                                              phsen_abcdef_signal_intensity_578_dim_0: 23,
                                              spectrum: 92,
                                              phsen_abcdef_signal_intensity_434_dim_0: 23,
                                              string5: 5, string18: 18,
                                              reference_light_measurements_dim_0: 16)
Coordinates:
  * spectrum                                 (spectrum) int32 368B 0 1 ... 90 91
  * reference_light_measurements_dim_0       (reference_light_measurements_dim_0) int32 64B ...
  * phsen_abcdef_signal_intensity_434_dim_0  (phsen_abcdef_signal_intensity_434_dim_0) int32 92B ...
  * phsen_abcdef_signal_intensity_578_dim_0  (phsen_abcdef_signal_intensity_578_dim_0) int32 92B ...
  * time                                     (time) float64 11kB 3.763e+09 .....
Dimensions without coordinates: string5, string18
Data variables: (12/24)
    phsen_abcdef_signal_intensity_578        (time, phsen_abcdef_signal_intensity_578_dim_0) float64 246kB ...
    phsen_battery_volts                      (time) float64 11kB 11.52 ... 11.56
    ph_seawater_qartod_results               (time) uint8 1kB 1 1 1 1 ... 1 1 1
    phsen_thermistor_temperature             (time) float64 11kB 11.05 ... 12.77
    record_type                              (time) uint8 1kB 10 10 10 ... 10 10
    ph_seawater_qartod_executed              (time) <U2 11kB '11' '11' ... '11'
    ...                                       ...
    ph_seawater_qc_results                   (time) uint8 1kB 29 29 29 ... 29 29
    ph_seawater_qc_executed                  (time) uint8 1kB 29 29 29 ... 29 29
    preferred_timestamp                      (time, string18) |S1 24kB b'i' ....
    record_time                              (time) uint32 5kB 0 0 0 0 ... 0 0 0
    reference_light_measurements             (time, reference_light_measurements_dim_0) int16 43kB ...
    unique_id                                (time) uint8 1kB 79 79 ... 135 135
Attributes: (12/70)
    node:                               RID26
    comment:                            
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  telemetered
    stream:                             phsen_abcdef_dcl_instrument
    ...                                 ...
    geospatial_lon_resolution:          0.1
    geospatial_vertical_units:          meters
    geospatial_vertical_resolution:     0.1
    geospatial_vertical_positive:       down
    lat:                                44.63558
    lon:                                -124.3039

In [10]:
# Extracting specific parts from the xarray dataset; convert the time to 
# python datetime (the time coordinate is in units of seconds since 
# 1900-01-01 so we're converting it to something more usable for analysis.)
ph_seawater = ds['ph_seawater']
time = ds['time']
if time.attrs['units'] == 'seconds since 1900-01-01 0:0:0':
    time_py = ooi_seconds_to_datetime(ds['time'].data)
